In [1]:
import numpy as np
from bert4keras.backend import keras, K
from bert4keras.models import build_transformer_model
from bert4keras.tokenizers import Tokenizer
from bert4keras.optimizers import Adam
from bert4keras.snippets import sequence_padding, DataGenerator
from bert4keras.snippets import open, ViterbiDecoder, to_array
from bert4keras.layers import ConditionalRandomField
from keras.layers import Dense
from keras.models import Model
from tqdm import tqdm

maxlen = 256
epochs = 1
batch_size = 32
bert_layers = 12
learning_rate = 2e-5  # bert_layers越小，学习率应该要越大
crf_lr_multiplier = 1000  # 必要时扩大CRF层的学习率
categories = set()

# bert配置
# 下载预训练权重：https://github.com/google-research/bert
config_path = 'chinese_L-12_H-768_A-12/bert_config.json'
checkpoint_path = 'chinese_L-12_H-768_A-12/bert_model.ckpt'
dict_path = 'chinese_L-12_H-768_A-12/vocab.txt'

'''
数据样例：
客$$$@@@O
服$$$@@@O
：$$$@@@O
2$$$@@@B-ODR
2$$$@@@I-ODR
1$$$@@@I-ODR
3$$$@@@I-ODR
7$$$@@@I-ODR
6$$$@@@I-ODR
4$$$@@@I-ODR
4$$$@@@I-ODR
2$$$@@@I-ODR
3$$$@@@I-ODR
1$$$@@@I-ODR
7$$$@@@I-ODR
加工：
['客服：221376442317', [3, 14, 'ODR']]
'''
def load_data(filename):
    """加载数据
    单条格式：[text, (start, end, label), (start, end, label), ...]，
              意味着text[start:end + 1]是类型为label的实体。
    """
    D = []
    with open(filename, encoding='utf-8') as f:
        f = f.read()
        for l in f.split('\n\n'):
            if not l:
                continue
            d = ['']
            for i, c in enumerate(l.split('\n')):
                char, flag = c.split('$$$@@@')
                d[0] += char
                if flag[0] == 'B':
                    d.append([i, i, flag[2:]])
                    categories.add(flag[2:])
                elif flag[0] == 'I':
                    d[-1][1] = i
            D.append(d)
    return D


# 标注数据
train_data = load_data('fapiao/train_data.txt')
valid_data = load_data('fapiao/dev_data.txt')
categories = list(sorted(categories))

# 建立分词器
tokenizer = Tokenizer(dict_path, do_lower_case=True)

class data_generator(DataGenerator):
    """数据生成器
    """
    def __iter__(self, random=False):
        batch_token_ids, batch_segment_ids, batch_labels = [], [], []
        for is_end, d in self.sample(random):
            tokens = tokenizer.tokenize(d[0], maxlen=maxlen)
            mapping = tokenizer.rematch(d[0], tokens)
            start_mapping = {j[0]: i for i, j in enumerate(mapping) if j}
            end_mapping = {j[-1]: i for i, j in enumerate(mapping) if j}
            token_ids = tokenizer.tokens_to_ids(tokens)
            segment_ids = [0] * len(token_ids)
            labels = np.zeros(len(token_ids))
            for start, end, label in d[1:]:
                if start in start_mapping and end in end_mapping:
                    start = start_mapping[start]
                    end = end_mapping[end]
                    labels[start] = categories.index(label) * 2 + 1
                    labels[start + 1:end + 1] = categories.index(label) * 2 + 2
            batch_token_ids.append(token_ids)
            batch_segment_ids.append(segment_ids)
            batch_labels.append(labels)
            if len(batch_token_ids) == self.batch_size or is_end:
                batch_token_ids = sequence_padding(batch_token_ids)
                batch_segment_ids = sequence_padding(batch_segment_ids)
                batch_labels = sequence_padding(batch_labels)
                yield [batch_token_ids, batch_segment_ids], batch_labels
                batch_token_ids, batch_segment_ids, batch_labels = [], [], []


"""
后面的代码使用的是bert类型的模型，如果你用的是albert，那么前几行请改为：
model = build_transformer_model(
    config_path,
    checkpoint_path,
    model='albert',
)
output_layer = 'Transformer-FeedForward-Norm'
output = model.get_layer(output_layer).get_output_at(bert_layers - 1)
"""

model = build_transformer_model(
    config_path,
    checkpoint_path,
)

output_layer = 'Transformer-%s-FeedForward-Norm' % (bert_layers - 1)
output = model.get_layer(output_layer).output
output = Dense(len(categories) * 2 + 1)(output)
CRF = ConditionalRandomField(lr_multiplier=crf_lr_multiplier)
output = CRF(output)

model = Model(model.input, output)
model.summary()

model.compile(
    loss=CRF.sparse_loss,
    optimizer=Adam(learning_rate),
    metrics=[CRF.sparse_accuracy]
)

class NamedEntityRecognizer(ViterbiDecoder):
    """命名实体识别器
    """
    def recognize(self, text):
        tokens = tokenizer.tokenize(text, maxlen=512)
        mapping = tokenizer.rematch(text, tokens)
        token_ids = tokenizer.tokens_to_ids(tokens)
        segment_ids = [0] * len(token_ids)
        token_ids, segment_ids = to_array([token_ids], [segment_ids])
        nodes = model.predict([token_ids, segment_ids])[0] #模型的输出
        labels = self.decode(nodes)
        entities, starting = [], False
        for i, label in enumerate(labels):
            if label > 0:
                if label % 2 == 1:
                    starting = True
                    entities.append([[i], categories[(label - 1) // 2]])
                elif starting:
                    entities[-1][0].append(i)
                else:
                    starting = False
            else:
                starting = False
        return [(mapping[w[0]][0], mapping[w[-1]][-1], l) for w, l in entities]

NER = NamedEntityRecognizer(trans=K.eval(CRF.trans), starts=[0], ends=[0])

def evaluate(data):
    """评测函数
    """
    X, Y, Z = 1e-10, 1e-10, 1e-10
    ['IVCN', 'TAXNO', 'TIL', 'ODR', 'PROV','CITY','COUNTY']
    a,b,c,dd,e,f,g=1e-10, 1e-10, 1e-10, 1e-10, 1e-10, 1e-10, 1e-10
    aa,bb,cc,ddd,ee,ff,gg = 1e-10, 1e-10, 1e-10, 1e-10, 1e-10, 1e-10, 1e-10
    for d in tqdm(data, ncols=100):
        R = set(NER.recognize(d[0]))
        T = set([tuple(i) for i in d[1:]])
        for i in R:
            if i in T:
                if i[2] == 'IVCN':
                    a +=1
                elif i[2] == 'TAXNO':
                    b +=1
                elif i[2] == 'TIL':
                    c +=1
                elif i[2] == 'ODR':
                    dd +=1
                elif i[2] == 'PROV':
                    e +=1
                elif i[2] == 'CITY':
                    f +=1
                elif i[2] == 'COUNTY':
                    g +=1
                    
        for i in T:
            if i[2] == 'IVCN':
                aa +=1
            elif i[2] == 'TAXNO':
                bb +=1
            elif i[2] == 'TIL':
                cc +=1
            elif i[2] == 'ODR':
                ddd +=1
            elif i[2] == 'PROV':
                ee +=1
            elif i[2] == 'CITY':
                ff +=1
            elif i[2] == 'COUNTY':
                gg +=1
        
        X += len(R & T)
        Y += len(R)
        Z += len(T)
    f1, precision, recall ,ivcn,taxno,til,odr,prov,city,county= 2 * X / (Y + Z), X / Y, X / Z,a/aa,b/bb,c/cc,dd/ddd,e/ee,f/ff,g/gg
    return f1, precision, recall,ivcn,taxno,til,odr,prov,city,county

class Evaluator(keras.callbacks.Callback):
    """评估与保存
    """
    def __init__(self):
        self.best_val_f1 = 0
    def on_epoch_end(self, epoch, logs=None):
        trans = K.eval(CRF.trans)
        NER.trans = trans
        f1, precision, recall,ivcn,taxno,til,odr,prov,city,county = evaluate(valid_data)
        # 保存最优
#         if f1 >= self.best_val_f1:
        self.best_val_f1 = f1
        model.save_weights(str(epoch)+'best_model.weights')
        print('valid:  f1: %.5f, precision: %.5f, recall: %.5f, best f1: %.5f,ivcn: %.5f, taxno: %.5f, til: %.5f,odr: %.5f, prov: %.5f, city: %.5f, county: %.5f\n' %
            (f1, precision, recall, self.best_val_f1,ivcn,taxno,til,odr,prov,city,county))

if False:
    evaluator = Evaluator()
    train_generator = data_generator(train_data, batch_size)

    model.fit(
        train_generator.forfit(),
        steps_per_epoch=len(train_generator),
        epochs=epochs,
        callbacks=[evaluator]
    )
else:
    model.load_weights('0best_model.weights')
    NER.trans = K.eval(CRF.trans)

Using TensorFlow backend.






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.







Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, None)         0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
Embedding-Token (Embedding)     (None, None, 768)    16226304    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, None, 768)    1536        

In [2]:
NER.trans.shape

(15, 15)

In [3]:
# ['客服：221376442317', [3, 14, 'ODR']]
NER.recognize('客服：221376442317')

[(3, 14, 'ODR')]

# 模型文件格式转换（h5->pb）

In [3]:
from keras import backend as K
import tensorflow as tf
# convert .h5 to .pb
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def,
                                                      output_names, freeze_var_names)
        return frozen_graph

print('input is :', (model.input[0].name,model.input[1].name))
print ('output is:', model.output.name)
sess = K.get_session()
frozen_graph = freeze_session(K.get_session(), output_names=[model.output.op.name])

input is : ('Input-Token:0', 'Input-Segment:0')
output is: conditional_random_field_1/add:0
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 205 variables.
INFO:tensorflow:Converted 205 variables to const ops.


In [4]:
from tensorflow.python.framework import graph_io

output_path='./pb'
pb_model_name='ner_model.pb'
# pb_model_name='t5/t5_model.pb'
graph_io.write_graph(frozen_graph, output_path, pb_model_name, as_text=False)

'./pb/ner_model.pb'

In [5]:
from tensorflow.python.saved_model import signature_constants
from tensorflow.python.saved_model import tag_constants
# export_dir = 't5/saved_model'
# # graph_pb = './5_trained_model.pb'
# graph_pb = 't5/t5_model.pb'
export_dir = './saved_model1'
# graph_pb = './5_trained_model.pb'
graph_pb = './pb/ner_model.pb'
builder = tf.saved_model.builder.SavedModelBuilder(export_dir)
with tf.gfile.GFile(graph_pb, "rb") as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())

In [6]:
sigs = {}
with tf.Session(graph=tf.Graph()) as sess:
    tf.import_graph_def(graph_def, name="")
    g = tf.get_default_graph()
    inp0 = g.get_tensor_by_name(model.input[0].name)
    inp1 = g.get_tensor_by_name(model.input[1].name)
    out = g.get_tensor_by_name(model.output.name)
    print('inp0',inp0)
    print('inp1',inp1)
    print('out',out)
    
    sigs[signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY] = tf.saved_model.signature_def_utils.predict_signature_def({"in0":inp0,"in1":inp1},{"out":out})
    builder.add_meta_graph_and_variables(sess,[tag_constants.SERVING],signature_def_map=sigs)
builder.save()

inp0 Tensor("Input-Token:0", shape=(?, ?), dtype=float32)
inp1 Tensor("Input-Segment:0", shape=(?, ?), dtype=float32)
out Tensor("conditional_random_field_1/add:0", shape=(?, ?, 15), dtype=float32)
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ./saved_model1/saved_model.pb


b'./saved_model1/saved_model.pb'